In [1]:
from __future__ import print_function, division
import argparse
import os
from os import listdir
from os.path import isfile, join

import math

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

from skimage import io, transform

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Subset, Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
torch.cuda.empty_cache()
plt.ion()   # interactive mode
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

In [2]:
def get_target_label_idx(labels, targets, shots=5, test=False):
    """
    Get the indices of labels that are included in targets.
    :param labels: array of labels
    :param targets: list/tuple of target labels
    :return: list with indices of target labels
    """
    final_list = []
    
    for t in targets:
        if test:
            final_list += np.argwhere(np.isin(labels, t)).flatten().tolist()
        else:
            final_list += np.argwhere(np.isin(labels, t)).flatten().tolist()[:shots]
    
    return final_list

# function to convert labels [5,6,7,8,9] to [0,1,2,3,4]
def convert_label(x):

    if x >= 5:
        return x-5
    else:
        return x

In [20]:
from torch.utils.data import Subset
from PIL import Image
from torchvision.datasets import MNIST

import torchvision.transforms as transforms

class ISIC_Dataset(Dataset):
    def __init__(self, root: str, normal_class=0):
        super().__init__()

#         self.n_classes = 2  # 0: normal, 1: outlier
#         self.normal_classes = tuple([1,2,3,4,5,6,7,8])
#         self.outlier_classes = list(range(9,10))

        #self.outlier_classes.remove([0,1,2,3])

        self.normal_classes = tuple([normal_class])
#         self.outlier_classes = list(range(0,8))
#         self.outlier_classes.remove(normal_class)

#         print('Normal class:', self.normal_classes)
#         print('outliers: ', self.outlier_classes)
        self.csv_file = '/home/fnunnari/skincaredata/ISIC_Challenge_2019/ISIC_2019_Training_GroundTruth.csv'

#         self.test_csv_file = '/home/haal01/home/haal01/final project srv/Deep SVDD/src/datasets/mscoco.csv'

#         self.test_root_dir = '/home/haal01/home/haal01/final project srv/mscoco2017' #mscoco


# tensor(13.6909) tensor(-26.0172)
# tensor(36.7479) tensor(-21.8755)
        
        train_min_max = [(-26.0172, 13.6909)]
        test_min_max = [( -21.8755, 36.7479)]
        nevus_min_max = [(-24.9848, 9.1327)]

        selected_min_max = nevus_min_max
        print('train min max:', train_min_max)
        print('test min max:', test_min_max)
        print('Selected min max:', test_min_max)

        print("Changed to nevus min max")
        # MNIST preprocessing: GCN (with L1 norm) and min-max feature scaling to [0,1]
#         transform = transforms.Compose([
#                                         transforms.ToTensor(),
#                                         transforms.Lambda(lambda x: global_contrast_normalization(x, scale='l1')),
#                                         transforms.Normalize([selected_min_max[0][0]] * 3,
#                                                              [selected_min_max[0][1] - selected_min_max[0][0]] * 3)])
        
#         test_transform = transforms.Compose([
#                                         transforms.ToTensor(),
#                                         transforms.Lambda(lambda x: global_contrast_normalization(x, scale='l1')),
#                                         transforms.Normalize([selected_min_max[0][0]] * 3,
#                                                              [selected_min_max[0][1] - selected_min_max[0][0]] * 3)])
        
    
        transform = transforms.Compose([
                                        transforms.ToTensor()])
        
        test_transform = transforms.Compose([
                                        transforms.ToTensor()])

#         target_transform = transforms.Lambda(lambda x: int(x in self.outlier_classes))
    
#         self.coco_outlier_classes = list(range(9,10))
#         coco_target_transform = transforms.Lambda(lambda x: int(x in self.coco_outlier_classes))

        self.train_set = ISIC(csv_file=self.csv_file, train=True, root_dir=root, transform=transform)
        self.test_set = ISIC(csv_file=self.csv_file, train=False, root_dir=root, transform=test_transform) # for unk class
        
       
#         train_label_list = []
        
#         train_idx_normal = get_target_label_idx(self.train_set.train_labels, self.normal_classes)
#         self.train_set = Subset(self.train_set, train_idx_normal)
#        # print(len(self.train_set))

#         self.test_set = ISIC(csv_file=self.test_csv_file, train=False, root_dir=self.test_root_dir, transform=test_transform, target_transform=target_transform) # for unk class
#         self.test_set = ISIC(csv_file=self.csv_file, train=False, root_dir=root, transform=transform, target_transform=target_transform)
#         self.coco_test_set = ISIC_COCO(csv_file=self.test_csv_file, train=False, root_dir=self.test_root_dir, transform=test_transform, target_transform=coco_target_transform) # for unk class


    def __len__(self):
        return len(self.total_dataset.target)


    

class ISIC(Dataset):

    def __init__(self, csv_file, root_dir, train=True, transform=None, target_transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """

        self.csv_data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.target = []
        self.test_split = 0.3
        self.split_len = int((1 - self.test_split) * len(self.csv_data))

     #   if train:
        #    for i in range(self.split_len):
           #     for j in range(1,9):
         #           if self.csv_data.iloc[i,j] == 1:
           #             self.target.append(j)


        if train:
            for i in range(1, self.split_len): #20000
#             for i in range(2000, 4000): #20000
                for j in range(1,9): #change it to 10 for including outlier class
                    if self.csv_data.iloc[i,j] == 1:
                        self.target.append(j)
                        break
        else:
            for i in range(self.split_len, len(self.csv_data)): #len(self.csv_data)
#             for i in range(6000,7500): #len(self.csv_data)
                for j in range(1,9):
                    if self.csv_data.iloc[i,j] == 1:
                        self.target.append(j)
                        break
                if i == 3000:
                    break


    def __len__(self):
        return len(self.target)

    @property
    def train_labels(self):
        # warnings.warn("test_labels has been renamed targets")

        return self.target

    @property
    def test_labels(self):
        return self.target



    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()


        if 'ISIC' in self.csv_data.iloc[idx+1, 0]:
            img_name = os.path.join(self.root_dir, self.csv_data.iloc[idx+1, 0] + '.jpg')
        else:
            img_name = os.path.join(self.root_dir, self.csv_data.iloc[idx+1, 0] + '.jpg')



        pic = Image.open(img_name, mode='r')
        pic = pic.resize((224,224),Image.NEAREST)
        #img = np.array(pic.getdata()).reshape(pic.size[0], pic.size[1], 3)
        img = np.asarray(pic).copy()
        t = (224,224,3)
        if img.shape != t:
            print(img.shape)
            print(img_name)
        #print(img.shape)
        # print(type(img), img.shape)
        # img = img[0:32][0:32][:]
        img = Image.fromarray(img)

        # img = ImgByteArr.getvalue()


        target = self.target[idx]
        target = target -1
        
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        
#         print(target, img_name, idx)
        return img, target, idx



In [6]:
isic_dataset = ISIC_Dataset(root='/home/fnunnari/skincaredata/ISIC_Challenge_2019/ISIC_2019_Training_Input', normal_class=[0,1,2,3,4,5,6,7])
test_loader = DataLoader(dataset=isic_dataset.test_set, batch_size=16, num_workers=0, shuffle=False)
train_loader = DataLoader(dataset=isic_dataset.train_set, batch_size=16, num_workers=0, shuffle=True)
print(len(isic_dataset.train_set))
classes = {'MEL':0, 'NV':1, 'BCC':2, 'AK':3, 'BKL':4, 'DF':5, 'VASC':6, 'SCC':7}
c_0 = 0
c_1 = 0
c_2 = 0
c_3 = 0
c_4 = 0
c_5 = 0
c_6 = 0
c_7 = 0
i = 1
for data in train_loader:
    img, target, idx = data
    
    for tg in target:
        if tg == 0:
           c_0 += 1
        elif tg == 1:
            c_1 += 1
        elif tg == 2:
            c_2 += 1
        elif tg == 3:
            c_3 += 1
        elif tg == 4:
            c_4 += 1
        elif tg == 5:
            c_5 += 1
        elif tg == 6:
            c_6 += 1
        elif tg == 7:
            c_7 ++ 1
    print("iter: ", i)
    i+=1
            
print(c_0, c_1, c_2, c_3,c_4,c_5,c_6,c_7)

train min max: [(-26.0172, 13.6909)]
test min max: [(-21.8755, 36.7479)]
Selected min max: [(-21.8755, 36.7479)]
Changed to nevus min max
2000
iter:  1
iter:  2
iter:  3
iter:  4
iter:  5
iter:  6
iter:  7
iter:  8
iter:  9
iter:  10
iter:  11
iter:  12
iter:  13
iter:  14
iter:  15
iter:  16
iter:  17
iter:  18
iter:  19
iter:  20
iter:  21
iter:  22
iter:  23
iter:  24
iter:  25
iter:  26
iter:  27
iter:  28
iter:  29
iter:  30
iter:  31
iter:  32
iter:  33
iter:  34
iter:  35
iter:  36
iter:  37
iter:  38
iter:  39
iter:  40
iter:  41
iter:  42
iter:  43
iter:  44
iter:  45
iter:  46
iter:  47
iter:  48
iter:  49
iter:  50
iter:  51
iter:  52
iter:  53
iter:  54
iter:  55
iter:  56
iter:  57
iter:  58
iter:  59
iter:  60
iter:  61
iter:  62
iter:  63
iter:  64
iter:  65
iter:  66
iter:  67
iter:  68
iter:  69
iter:  70
iter:  71
iter:  72
iter:  73
iter:  74
iter:  75
iter:  76
iter:  77
iter:  78
iter:  79
iter:  80
iter:  81
iter:  82
iter:  83
iter:  84
iter:  85
iter:  86
iter: 

In [25]:
classes = {'MEL':0, 'NV':1, 'BCC':2, 'AK':3, 'BKL':4, 'DF':5, 'VASC':6, 'SCC':7}
c_0 = 0
c_1 = 0
c_2 = 0
c_3 = 0
c_4 = 0
c_5 = 0
c_6 = 0
c_7 = 0
i = 1
for data in test_loader:
    img, target, idx = data
    
    for tg in target:
        if tg == 0:
           c_0 += 1
        elif tg == 1:
            c_1 += 1
        elif tg == 2:
            c_2 += 1
        elif tg == 3:
            c_3 += 1
        elif tg == 4:
            c_4 += 1
        elif tg == 5:
            c_5 += 1
        elif tg == 6:
            c_6 += 1
        elif tg == 7:
            c_7 ++ 1
#     print("iter: ", i)
    i+=1
            
print(c_0, c_1, c_2, c_3,c_4,c_5,c_6,c_7)

iter:  1
iter:  2
iter:  3
iter:  4
iter:  5
iter:  6
iter:  7
iter:  8
iter:  9
iter:  10
iter:  11
iter:  12
iter:  13
iter:  14
iter:  15
iter:  16
iter:  17
iter:  18
iter:  19
iter:  20
iter:  21
iter:  22
iter:  23
iter:  24
iter:  25
iter:  26
iter:  27
iter:  28
iter:  29
iter:  30
iter:  31
iter:  32
iter:  33
iter:  34
iter:  35
iter:  36
iter:  37
iter:  38
iter:  39
iter:  40
iter:  41
iter:  42
iter:  43
iter:  44
iter:  45
iter:  46
iter:  47
iter:  48
iter:  49
iter:  50
iter:  51
iter:  52
iter:  53
iter:  54
iter:  55
iter:  56
iter:  57
iter:  58
iter:  59
iter:  60
iter:  61
iter:  62
iter:  63
iter:  64
iter:  65
iter:  66
iter:  67
iter:  68
iter:  69
iter:  70
iter:  71
iter:  72
iter:  73
iter:  74
iter:  75
iter:  76
iter:  77
iter:  78
iter:  79
iter:  80
iter:  81
iter:  82
iter:  83
iter:  84
iter:  85
iter:  86
iter:  87
iter:  88
iter:  89
iter:  90
iter:  91
iter:  92
iter:  93
iter:  94
127 1051 89 26 148 13 18 0


In [52]:

classes = {'MEL':0, 'NV':1, 'BCC':2, 'AK':3, 'BKL':4, 'DF':5, 'VASC':6, 'SCC':7}
c_0 = 0
c_1 = 0
c_2 = 0
c_3 = 0
c_4 = 0
c_5 = 0
c_6 = 0
c_7 = 0
i = 1
for data in train_loader:
    img, target, idx = data
    
    for tg in target:
        if tg == 0:
           c_0 += 1
        elif tg == 1:
            c_1 += 1
        elif tg == 2:
            c_2 += 1
        elif tg == 3:
            c_3 += 1
        elif tg == 4:
            c_4 += 1
        elif tg == 5:
            c_5 += 1
        elif tg == 6:
            c_6 += 1
        elif tg == 7:
            c_7 ++ 1
    print("iter: ", i)
    i+=1
            
print(c_0, c_1, c_2, c_3,c_4,c_5,c_6,c_7)

iter:  1
iter:  2
iter:  3
iter:  4
iter:  5
iter:  6
iter:  7
iter:  8
iter:  9
iter:  10
iter:  11
iter:  12
iter:  13
iter:  14
iter:  15
iter:  16
iter:  17
iter:  18
iter:  19
iter:  20
iter:  21
iter:  22
iter:  23
iter:  24
iter:  25
iter:  26
iter:  27
iter:  28
iter:  29
iter:  30
iter:  31
iter:  32
iter:  33
iter:  34
iter:  35
iter:  36
iter:  37
iter:  38
iter:  39
iter:  40
iter:  41
iter:  42
iter:  43
iter:  44
iter:  45
iter:  46
iter:  47
iter:  48
iter:  49
iter:  50
iter:  51
iter:  52
iter:  53
iter:  54
iter:  55
iter:  56
iter:  57
iter:  58
iter:  59
iter:  60
iter:  61
iter:  62
iter:  63
iter:  64
iter:  65
iter:  66
iter:  67
iter:  68
iter:  69
iter:  70
iter:  71
iter:  72
iter:  73
iter:  74
iter:  75
iter:  76
iter:  77
iter:  78
iter:  79
iter:  80
iter:  81
iter:  82
iter:  83
iter:  84
iter:  85
iter:  86
iter:  87
iter:  88
iter:  89
iter:  90
iter:  91
iter:  92
iter:  93
iter:  94
iter:  95
iter:  96
iter:  97
iter:  98
iter:  99
iter:  100
iter:  1

In [65]:
csv_data_file = pd.read_csv('/home/fnunnari/skincaredata/ISIC_Challenge_2019/ISIC_2019_Training_GroundTruth.csv')
csv_data_file.describe()


,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
count,25331.000000,25331.000000,25331.000000,25331.000000,25331.000000,25331.000000,25331.000000,25331.000000,25331.0
mean,0.178516,0.508270,0.131183,0.034227,0.103588,0.009435,0.009988,0.024792,0.0
std,0.382954,0.499941,0.337607,0.181815,0.304732,0.096677,0.099440,0.155493,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [8]:
class VggModel(nn.Module):
    def __init__(self, n_class, fine_tune=None, pretrained=True):
        super(VggModel, self).__init__()
        #################################################################################
        # TODO: Build the classification network described in Q4 using the              #
        # models.vgg11_bn network from torchvision model zoo as the feature extraction  #
        # layers and two linear layers on top for classification. You can load the      #
        # pretrained ImageNet weights based on the pretrained flag. You can enable and  #
        # disable training the feature extraction layers based on the fine_tune flag.   #
        #################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

#         vgg11_bn_model = models.vgg11_bn(pretrained=pretrained)

#         self.model = nn.Sequential()
#         self.model.features = vgg11_bn_model.features
# #         if fine_tune == False:
# #             for param in self.model.parameters():
# #                 param.requires_grad = False


#         self.model.classifier = nn.Sequential(
#             nn.Linear(in_features=512*7*7, out_features=2048, bias=True),
#             nn.BatchNorm1d(2048),
#             nn.ReLU(),
#             nn.Linear(in_features=2048, out_features=n_class, bias=True)
#         )
    
    
        vgg16Net = models.vgg16_bn(pretrained=True)
        modules = list(vgg16Net.children())[:-1]
        # for m in modules:
        #     print(m)


        self.model = nn.Sequential()
        self.model.features = nn.Sequential(*modules)
        # self.model.addConv = nn.Conv2d(512, self.rep_dim, 3, padding=1)
        # self.model.bn2daddConv = nn.BatchNorm2d(self.rep_dim)

        #         self.features = nn.Sequential(*list(vgg16Net.children())[:-1])

        self.model.classifier = nn.Sequential(
            nn.Linear(in_features=512*7*7, out_features=2048, bias=True),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Linear(in_features=2048, out_features=n_class, bias=True)
        )
    


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    def forward(self, x):
        #################################################################################
        # TODO: Implement the forward pass computations                                 #
        #################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        x = self.model.features(x)
        x = x.view(x.size(0), -1)
        x = self.model.classifier(x)
        out = x
#         out = out.squeeze()

#         out = self.model.classifier(out)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return out




In [9]:
torch.cuda.empty_cache()
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
torch.cuda.current_device()
# torch.cuda.device_count()

1

In [10]:

net= VggModel(8, fine_tune=None, pretrained=True)
net.to(device)

VggModel(
  (model): Sequential(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (9): ReLU(inplace=True)
        (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (12): ReLU(inplace=True)
     

In [11]:
# Initialize the model for this run

print(net)
from torchsummary import summary
summary(net, (3,224,224))

VggModel(
  (model): Sequential(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (9): ReLU(inplace=True)
        (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (12): ReLU(inplace=True)
     

In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    test_true_label = []
    test_pred_label = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            
            if phase == 'train':
                model.train()  # Set model to training mode
                
                running_loss = 0.0
                running_corrects = 0
                
                for data in train_loader:
                    inputs, labels, idx = data
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(True):
                        outputs = model(inputs)
                        
                        _, preds = torch.max(outputs, 1)
#                         print(preds.data, type(preds[0]), labels)
                        loss = criterion(outputs, labels)
                        
                        loss.backward()
                        optimizer.step()
                    
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    
                scheduler.step()
                
                epoch_loss = running_loss / train_size
                epoch_acc = running_corrects.double() / train_size

                print('{} Loss: {:.4f} Acc: {:.4f}'.format('Train', epoch_loss, epoch_acc))
                    
            elif phase == 'val' and epoch == (num_epochs -1):
                
                model.eval()   # Set model to evaluate mode
                
                running_loss = 0.0
                running_corrects = 0
                
                for data in test_loader:
                    inputs, labels, idx = data
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(False):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        
                        loss = criterion(outputs, labels)
                
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    test_true_label.append(labels.data)
                    test_pred_label.append(preds.data)
                
                epoch_loss = running_loss / test_size
                epoch_acc = running_corrects.double() / test_size
                
                print('{} Loss: {:.4f} Acc: {:.4f}'.format('Val', epoch_loss, epoch_acc))
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc, test_true_label, test_pred_label

In [21]:
isic_dataset = ISIC_Dataset(root='/home/fnunnari/skincaredata/ISIC_Challenge_2019/ISIC_2019_Training_Input', normal_class=[0,1,2,3,4,5,6,7])
test_loader = DataLoader(dataset=isic_dataset.test_set, batch_size=16, num_workers=2, shuffle=False)
train_loader = DataLoader(dataset=isic_dataset.train_set, batch_size=16, num_workers=2, shuffle=True)
print(len(isic_dataset.train_set))

train min max: [(-26.0172, 13.6909)]
test min max: [(-21.8755, 36.7479)]
Selected min max: [(-21.8755, 36.7479)]
Changed to nevus min max
17730


In [22]:
train_size =  len(isic_dataset.train_set)
test_size = len(isic_dataset.test_set)

# net = VggModel(8, fine_tune=None, pretrained=True).to(device)
# loading pretrained model
#     net = torch.load('LeNet_5_class_pretrained_model.tar')
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(net.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

best_net, best_acc, test_true, test_pred = train_model(net, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=12)

Epoch 0/11
----------
Train Loss: 1.0888 Acc: 0.6210

Epoch 1/11
----------
Train Loss: 1.0181 Acc: 0.6420

Epoch 2/11
----------
Train Loss: 0.9899 Acc: 0.6507

Epoch 3/11
----------
Train Loss: 0.9515 Acc: 0.6597

Epoch 4/11
----------
Train Loss: 0.9127 Acc: 0.6699

Epoch 5/11
----------
Train Loss: 0.8896 Acc: 0.6776

Epoch 6/11
----------
Train Loss: 0.8814 Acc: 0.6804

Epoch 7/11
----------
Train Loss: 0.8047 Acc: 0.7042

Epoch 8/11
----------
Train Loss: 0.7614 Acc: 0.7187

Epoch 9/11
----------
Train Loss: 0.7404 Acc: 0.7243

Epoch 10/11
----------
Train Loss: 0.7215 Acc: 0.7324

Epoch 11/11
----------
Train Loss: 0.6999 Acc: 0.7356
Val Loss: 4.3186 Acc: 0.2836

Training complete in 59m 58s
Best val Acc: 0.283553


In [25]:
torch.save(best_net.state_dict(), 'base_model1.pt')

In [23]:
l_true = []
l_pred = []
for d in test_true:
    l_list = d.cpu().numpy()
    for l in l_list:
        l_true.append(l)
print(l_true)
for d in test_pred:
    l_list = d.cpu().numpy()
    for l in l_list:
        l_pred.append(l)

[2, 2, 2, 1, 1, 7, 1, 0, 0, 2, 0, 0, 7, 0, 0, 7, 1, 2, 2, 2, 2, 2, 2, 1, 2, 4, 1, 2, 1, 1, 0, 4, 1, 0, 5, 2, 4, 1, 7, 2, 7, 1, 4, 7, 0, 3, 0, 0, 0, 2, 0, 1, 0, 7, 0, 1, 1, 2, 1, 1, 1, 6, 2, 0, 6, 0, 2, 2, 3, 1, 4, 2, 1, 1, 0, 3, 1, 1, 3, 0, 2, 1, 2, 1, 5, 3, 2, 2, 1, 2, 0, 2, 1, 0, 4, 7, 2, 1, 1, 2, 1, 1, 3, 0, 4, 0, 2, 4, 2, 1, 2, 0, 2, 2, 1, 1, 0, 2, 0, 4, 0, 2, 3, 0, 4, 0, 1, 3, 0, 0, 1, 0, 3, 2, 7, 4, 1, 2, 1, 2, 1, 2, 1, 0, 4, 0, 1, 3, 0, 1, 4, 0, 2, 1, 4, 4, 1, 2, 2, 2, 2, 2, 0, 0, 0, 2, 7, 1, 0, 2, 1, 1, 3, 2, 1, 2, 2, 1, 2, 1, 0, 4, 1, 0, 2, 2, 2, 1, 2, 0, 5, 0, 2, 3, 1, 1, 0, 4, 2, 1, 0, 0, 2, 2, 4, 2, 0, 0, 1, 4, 1, 0, 0, 0, 1, 4, 1, 1, 2, 2, 0, 1, 1, 2, 0, 4, 1, 0, 0, 2, 0, 6, 1, 0, 1, 1, 1, 1, 1, 0, 1, 2, 1, 0, 2, 0, 0, 3, 0, 1, 2, 3, 4, 1, 3, 1, 0, 2, 7, 2, 1, 2, 5, 0, 1, 4, 6, 1, 2, 1, 2, 1, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 2, 1, 4, 1, 0, 4, 0, 1, 1, 1, 1, 4, 4, 0, 2, 0, 3, 1, 0, 2, 1, 1, 6, 1, 2, 4, 0, 1, 2, 0, 0, 0, 3, 4, 4, 7, 3, 4, 2, 2, 4, 0, 2, 0, 1, 2, 0, 1, 

In [24]:
from sklearn.metrics import classification_report

target_names = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']
print(classification_report(l_true, l_pred, target_names=target_names))

              precision    recall  f1-score   support

         MEL       0.21      0.10      0.13      1709
          NV       0.34      0.70      0.46      2603
         BCC       0.24      0.05      0.08      1715
          AK       0.08      0.01      0.02       457
         BKL       0.10      0.11      0.10       710
          DF       0.00      0.00      0.00        76
        VASC       0.00      0.00      0.00        76
         SCC       0.04      0.02      0.03       254

   micro avg       0.28      0.28      0.28      7600
   macro avg       0.13      0.12      0.10      7600
weighted avg       0.23      0.28      0.22      7600



In [ ]:


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters, lr=learning_rate, weight_decay=reg)

# Train the model
lr = learning_rate
total_step = len(train_loader)
best_score = 0.0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Code to update the lr
    lr *= learning_rate_decay
    update_lr(optimizer, lr)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        #################################################################################
        # TODO: Q2.b Use the early stopping mechanism from previous questions to save   #
        # the model which has acheieved the best validation accuracy so-far.            #
        #################################################################################
        #best_model = None
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        if correct / total > best_score:
            best_model = model
            best_score = correct / total
        #best_model = model.copy()
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


        print('Validataion accuracy is: {} %'.format(100 * correct / total))
